#   Crime Classification

introduction 


## Importation de notre base de donnée, analyse et data preprocessing



#### Importation de notre base de données

- Remarque : Ajout d'un "r" dans le code pour transformer notre chaîne de caractères en "raw string" afin de permettre une lecture de notre chaîne de caractères sans prendre en compte les séquences d'échappement (ex : "\n", "\t", et dans notre cas "\U" (représente les caractères Unicode)), afin d'éviter toutes erreurs d'importations.

- Une autre solution aurait été de renommer "Users" pour éviter ce problème..

In [1]:
# importation du module pandas
import pandas as pd

# Importation de la base de données 
crime = pd.read_csv(r"C:\Users\mamad\Desktop\Projet_Pyhton\donnee\Crime_Data_from_2020_to_Present_20240416.csv")

# Affichage des 5 premières lignes
crime.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\mamad\\Desktop\\Projet_Pyhton\\donnee\\Crime_Data_from_2020_to_Present_20240416.csv'

#### Information sur notre base de données (type des données, nom de chaque colonne , nombre de lignes et de colonne)

In [ ]:

crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925720 entries, 0 to 925719
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           925720 non-null  int64  
 1   Date Rptd       925720 non-null  object 
 2   DATE OCC        925720 non-null  object 
 3   TIME OCC        925720 non-null  int64  
 4   AREA            925720 non-null  int64  
 5   AREA NAME       925720 non-null  object 
 6   Rpt Dist No     925720 non-null  int64  
 7   Part 1-2        925720 non-null  int64  
 8   Crm Cd          925720 non-null  int64  
 9   Crm Cd Desc     925720 non-null  object 
 10  Mocodes         796258 non-null  object 
 11  Vict Age        925720 non-null  int64  
 12  Vict Sex        802562 non-null  object 
 13  Vict Descent    802552 non-null  object 
 14  Premis Cd       925709 non-null  float64
 15  Premis Desc     925161 non-null  object 
 16  Weapon Used Cd  319701 non-null  float64
 17  Weapon Des

## Data preprocessing  

Nous allons nous concentrer ici sur la préparation de notre ensemble de données criminelles pour l'analyse.

### Gestion des valeurs manquantes

- Explication du code : .isna permet de retourner une base de données avec les valeurs True ("est une valeur manquante") et False ("n'est pas une valeur manquante").

- Pour rappel, True = 1 et False = 0. Donc lorsque l'on fait sum (somme d'une séquence de données) après .isna, Python calcule la somme totale des valeurs manquantes pour chaque colonne.

- Détail : on aura une séquence de 1 et de 0 pour chaque colonne qui correspondent aux valeurs manquantes ou non (rôle de la fonction isna) et on additionne les valeurs de chacune des séquences (rôle de la fonction sum) pour avoir le nombre de valeurs manquantes dans chacune des colonnes.


In [ ]:
# voir les valeurs manquantes pour chaque colonne de notre base de données

crime.isna().sum()


DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           129462
Vict Age               0
Vict Sex          123158
Vict Descent      123168
Premis Cd             11
Premis Desc          559
Weapon Used Cd    606019
Weapon Desc       606019
Status                 0
Status Desc            0
Crm Cd 1              11
Crm Cd 2          858483
Crm Cd 3          923460
Crm Cd 4          925656
LOCATION               0
Cross Street      780494
LAT                    0
LON                    0
dtype: int64

### Imputation de valeurs pour la colonne 'Mocodes': 

- .fillna permet de remplacer une valeur manquante par une valeur que l'on aura spécifiée (imputation). Ici, on remplace les valeurs manquantes de la colonne 'Mocodes' par '1501 Other MO'.

- On met inplace=True car on souhaite modifier les valeurs directement dans notre base de données sans créer de copie. Normalement, Python crée une copie de cette base lorsqu'il y a des modifications (pour conserver la base originale), ce qui utilise plus de mémoire.

In [ ]:
#Nettoyage des données

crime["Mocodes"].fillna("1501 Other MO", inplace=True)

#### Imputation de valeurs pour les colonnes mentionnant le sexe de la victime et son ethnie
- On procède de la même manière que pour les Mocodes et on impute cette fois-ci un 'X' pour les colonnes 'Vict Sex' et 'Vict Descent'.

In [ ]:
crime['Vict Sex'].fillna('X', inplace=True)
crime['Vict Descent'].fillna('X', inplace=True)

####    imputation de valeurs pour les colone mentionnant le type d'arme 
- Pour les colonnes 'Weapon Used Cd' (qui contiennent les types d'armes associés à des codes juridiques dans la loi) et 'Weapon Desc' (qui contient la description du code associé à l'arme utilisée, donc si la troisième ligne de Weapon Used Cd = 101, alors la troisième ligne de Weapon Desc sera la description du code 101), on impute la valeur '500' dans la colonne 'Weapon Used Cd', qui est le code lorsque l'arme est inconnue. De plus, on impute la valeur 'UNKNOWN WEAPON/OTHER WEAPON' dans la colonne Weapon Desc, qui est la description du code 500, conformément aux codes de la juridiction de Los Angeles.

In [ ]:
# on impute la description du code 500 dans la colonne qui decrit ces codes
crime['Weapon Desc'].fillna('UNKNOWN WEAPON/OTHER WEAPON', inplace=True)

#assigne la valeur 500 lorsque la valeur est manquante

crime["Weapon Used Cd"].fillna(500, inplace=True)



### Suppression des données inutiles


#### Supprimer les lignes qui ne contiennent pas les valeurs de la colonne 'Premis Desc'

- Explication du code : `dropna` permet de supprimer les valeurs manquantes. Cette suppression implique la suppression des lignes où il y a des valeurs manquantes. En effectuant plusieurs recherches sur des forums, j'ai découvert une fonction appelée `subset` qui m'a permis de spécifier sur quelle colonne `dropna` devrait être appliqué.

- L'importance d'utiliser cela au lieu de `crime['Premis Desc'].dropna()` est que cette approche permet de modifier les valeurs d'une colonne dans le DataFrame original, ce qui permet d'afficher la colonne modifiée avec les autres colonnes. L'utilité de `crime['Premis Desc'].dropna` est lorsque l'on veut seulement étudier cette colonne en particulier car notre code nous renvoie dans le terminal seulement la colonne 'Premis Desc' modifiée. En réalité, le DataFrame n'a pas été modifié car Python a créé une copie de la colonne 'Premis Desc' pour pouvoir appliquer ces modifications.

In [ ]:

crime = crime.dropna(subset=['Premis Desc'])


#### Suppression des colonnes

- Explication du code : Nous utilisons la méthode `.drop()` pour supprimer nos colonnes. L'argument `inplace=True` permet de supprimer les colonnes directement dans notre base de données `crime` de manière définitive (les colonnes ne doivent pas réapparaître lorsque nous appelons notre base) et `axis=1` permet de spécifier que nous voulons supprimer les colonnes.

- Remarque : Nous aurions également pu faire `crime.drop(crime['Crm Cd 1'], inplace=True)`, le résultat aurait été similaire.

In [ ]:
# supprimer la colonne Part 1-2

crime.drop(crime["Part 1-2"], inplace=True)

#supprimer la colonne Crm Cd 1
crime.drop('Crm Cd 1', axis=1, inplace=True)

# supprimer la colonne Crm Cd 2
crime.drop('Crm Cd 2', axis=1, inplace=True)

#supprimer la colonne Crm Cd 3
crime.drop('Crm Cd 3', axis=1, inplace=True)

#supprimer la colonne Crm Cd 4
crime.drop('Crm Cd 4', axis=1, inplace=True)

#### filtrage des données : Convertir la colonne date en objet Date


-   On va convertir la colonne Date Rptd en format datetime pour faciliter l'analyse temporelle. Les objets datetime sont très utile car elle permette de stocker des élèments temporelles ce qui facilite les opérations que l'on peut faire sur des données temporelles ( par exemple :  analyser les crimes seulement après 18 heures, les crimes qui commence pendant les 3 premiers jours de chaque mois... ). L'objet Datetime permet aussi de séparer les donnée selon un type temporelle ( année , jours , heures, mois)

-   Mais pour notre analyse, la convertion en datetime ne suffit pas : On va aussi separé les dates, les années et les mois en differentes colonnes pour faciliter le filtrage des données :   On pourra grâce à cela regrouper les données selon les mois, les années et même les jours ce qui est très utile pour analyser certaines tendances.

- explication du code : pd.to_datetime permet de convertir la colonne Date Rptd en format datetime. En regardant un peu les fonctions d'analyse temporelle sur le site de pandas, on a trouvé une fonction qui se nomme .dt et qui permet d'extraire les données temporelles que l'on souhaite. Le .year , month ou day  permet de spécifier qu'elle type de donnée temporelle on souhaite extraire grâce au .dt . Donc ici, on crée des colonnes Année, mois et jours qui contiendrons respectivement les années, les mois et les jours de chaque crime.



In [ ]:
#convertion en objet datetime
crime['Date Rptd'] = pd.to_datetime(crime['Date Rptd'])

# Création de nouvelle colonne

crime['Année'] = crime['Date Rptd'].dt.year

crime['Mois'] = crime['Date Rptd'].dt.month

crime['Jours'] = crime['Date Rptd'].dt.day

C:\Users\mamad\AppData\Local\Temp\ipykernel_25616\1078158148.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crime['Date Rptd'] = pd.to_datetime(crime['Date Rptd'])


#### Renommer les colonnes

- Nous avons renommé les colonnes pour qu'elles suivent les conventions de Python (avec le '_') et éviter d'éventuelles erreurs.

- Explication du code : Nous créons un dictionnaire qui contient toutes les clés (les anciens noms de colonnes) et les valeurs associées (les nouveaux noms de colonnes). Ensuite, nous utilisons la méthode `.rename()` pour renommer les colonnes comme spécifié entre parenthèses. En affectant le résultat à `columns` (les colonnes de notre DataFrame actuel), nous modifions directement notre DataFrame.

In [ ]:
#renommer les colonnes

new_column_names = {'Date Rptd': 'date_rptd', 
                    'DATE OCC': 'date_occ', 
                    'TIME OCC': 'time_occ', 
                    'AREA NAME': 'area_name', 
                    'Rpt Dist No': 'rpt_dist_no', 
                    'Crm Cd': 'crm_cd', 
                    'Crm Cd Desc': 'crm_cd_desc', 
                    'Vict Age': 'vict_age', 
                    'Vict Sex': 'vict_sex', 
                    'Vict Descent': 'vict_descent', 
                    'Premis Cd': 'premis_cd', 
                    'Premis Desc': 'premis_desc', 
                    'Weapon Used Cd': 'weapon_used_cd', 
                    'Weapon Desc': 'weapon_desc', 
                    'Status Desc': 'status_desc',  
                    'Cross Street': 'cross_street'
                   }

crime.rename(columns=new_column_names, inplace=True)


#### Catégoriser les Crimes grâce aux rapports de l'UCR

- Explication du code : Nous créons une liste pour chaque type de crime. Dans ces listes, nous avons des codes de crimes (les codes associés aux crimes commis). Notre objectif est de faire correspondre (mapper) les crimes que nous voyons dans notre base de données en catégories, conformément à ce qui est écrit dans le rapport de l'UCR. Par exemple, les codes de crime 110 (homicide) et 113 (homicide involontaire) représentent les crimes d'homicide. Nous allons donc les regrouper dans la catégorie homicide. Nous faisons ensuite la même chose pour les vols, les viols, les violences domestiques, etc.

- Remarque : Nous avons une catégorie 'Other theft' qui regroupe les crimes qui ne peuvent pas être regroupés dans d'autres catégories.

- De plus, nous devons créer une fonction pour pouvoir mapper les codes de crimes aux catégories. Dans cette fonction, nous spécifions notre paramètre (code) qui prendra les valeurs de la colonne 'crm cd' (codes de crimes). Nous indiquons que lorsque la valeur de code est similaire à un élément de homicide, de viol, de vol... nous retournons le nom de la catégorie associée au code de crime.

- Ensuite, nous appliquerons la fonction à la colonne 'crm cd' grâce à `.apply()`. Les catégories de crimes remplaceront donc leurs codes de crimes respectifs. Cela augmentera la lisibilité des données de notre base de données.

In [ ]:

# Definition des codes crime par rapport à chaque catégorie : Basé sur le rapport de l' UCR ==> pdf  UCR.COMPSTAT062618

# Homicide
HOMICIDE = [110, 113]

# Viol
RAPE = [121, 122, 815, 820, 821]

# Vol
ROBBERY = [210, 220]

# Assaults aggravated ==> aggression aggravé
AGG_ASSAULTS = [230, 231, 235]

# Violence domestique
DOMESTIC_VIOLENCE = [626, 627, 647, 763, 928, 930, 236, 250, 251, 761, 926]

# agression simple
SIMPLE_ASSAULT = [435, 436, 437, 622, 623, 624, 625]

# cambriolage
BURGLARY = [310, 320]

# Motor vehicule stolen ( ou GTA : grand theft auto) ==> vol de vehicule
MVT = [510, 520, 433]

# Burgalry / theft for vehicule
BTFV = [330, 331, 410, 420, 421]

# vol personnelle 
PERSONAL_THFT = [350, 351, 352, 353, 450, 451, 452, 453]


# fonction qui va nous permettre de mapper les codes crimes de notre base de donnée aux catégories que l'on a crée au dessus
def map_category(code):
    if code in HOMICIDE:
        return 'HOMICIDE'
    elif code in RAPE:
        return 'RAPE'
    elif code in ROBBERY:
        return 'ROBBERY'
    elif code in AGG_ASSAULTS:
        return 'AGG.ASSAULTS'
    elif code in DOMESTIC_VIOLENCE:
        return 'Domestic.Violence'
    elif code in SIMPLE_ASSAULT:
        return 'SIMPLE.ASSAULT'
    elif code in BURGLARY:
        return 'BURGLARY'
    elif code in MVT:
        return 'MVT'
    elif code in BTFV:
        return 'BTFV'
    elif code in PERSONAL_THFT:
        return 'PERSONAL.THFT'
    else:
        return 'OTHER.THEFT'

# Mapper les codes crimes aux catégories

if 'crm_cd' in crime.columns:
   
    crime['crm_cd'] = crime['crm_cd'].apply(map_category)
else:
    print("The 'Crm Cd' column does not exist in the DataFrame.")

#### Catégoriser les crimes selon leur nature

- Nous créons un dictionnaire pour relier chaque catégorie de crimes à une catégorie plus générale. Par exemple, les catégories qui représentent des crimes liés à des vols seront catégorisées dans la catégorie "vol". Nous choisissons le mot "larceny" pour ne pas trop nous répéter avec le mot "theft". Les homicides ne sont ni des agressions au sens strict de la loi, ni des vols, donc leur catégorisation sera "inconnue".

- Ensuite, nous créons une fonction qui permettra d'associer chaque catégorie de la colonne 'crm_cd' à sa catégorie plus générale qui lui est associée. Si la valeur de 'crm_cd' correspond à une clé du dictionnaire, la fonction retourne la valeur correspondante à cette clé (par exemple, si 'crm_cd' = 'BFTV', alors `crime_dict["BFTV"]` correspond à la valeur "larceny").

- Enfin, nous écrivons une instruction pour exécuter notre fonction sur la colonne 'crm_cd'. Les catégories générales des catégories initiales seront dans une nouvelle colonne appelée "crime_category" (car nous avons besoin des valeurs de la colonne 'crm_cd' dans notre analyse).

In [ ]:

# Creation d'un dictionaire 
crime_dict = {
    "BTFV": "Larceny",
    "BURGLARY": "Larceny",
    "ROBBERY": "Larceny",
    "MVT": "Larceny",
    "OTHER.THEFT": "Larceny",
    "PERSONAL.THFT": "Larceny",
    "RAPE": "Assault",
    "AGG.ASSAULTS": "Assault",
    "Domestic.Violence": "Assault",
    "SIMPLE.ASSAULT": "Assault",
    "HOMICIDE": "Unknown"
}

# Definition de notre fonction de correspondance
def categorize_crime(crm_cd):
    if crm_cd in crime_dict:
        return crime_dict[crm_cd]
    else:
        return None

# Appliquer la fonction pour la colonne  crm_cd et création d'une nouvelle colonne
crime["crime_category"] = crime["crm_cd"].apply(categorize_crime)


#### Trier les types de crime selon leur degré d'agressivité (Violent ou Property)

Il peut être pertinent de catégoriser les crimes selon s'ils sont violents ou s'ils ciblent seulement la propriété. En juridiction pénale à Los Angeles, on distingue ces deux types de crimes. Le document de l'UCR nous donne la liste des crimes qui appartiennent soit aux crimes violents, soit aux crimes contre la propriété. Cette distinction est cruciale car cela nous donne un bon indice sur la dangerosité d'un quartier, mais aussi très utile pour appliquer de nouvelles politiques et une allocation des ressources optimale (fournir plus de matériel aux agents de l'ordre afin d'être mieux préparé face aux criminels).

Les "violent crimes" (crimes violents) correspondent à des actes criminels où la force ou la menace est utilisée contre des victimes.

Les "property crimes" (ou crimes contre la propriété) font référence à une catégorie de crimes qui impliquent le vol de propriété ou la dégradation de biens, sans usage de force ou menace contre les victimes.

Nous avons utilisé la même méthode que pour la catégorisation plus générale des catégories des crimes. Cette fois-ci, nous avons créé une colonne "crime_type".

In [ ]:


# dictionnaire avec lels catégorie de crimes et leurs type d'agressivité.
crime_dict = {
    "BTFV": "Property",
    "MVT": "Property",
    "BURGLARY": "Property",
    "OTHER.THEFT": "Property",
    "PERSONAL.THFT": "Property",
    "HOMICIDE": "Violent",
    "RAPE": "Violent",
    "ROBBERY": "Violent",
    "AGG.ASSAULTS": "Violent",
    "Domestic.Violence": "Violent",
    "SIMPLE.ASSAULT": "Violent"
}

# fonction pour mapper les donnée
def categorize_crime_type(crm_cd):
    if crm_cd in crime_dict:
        return crime_dict[crm_cd]
    else:
        return None

# instruction pour apliquer la fonction dans la colonne crm_code et afficher les données dans une  une nouvelle colonne 
crime["crime_type"] = crime["crm_cd"].apply(categorize_crime_type)


Nous supprimons la colonne 'date_rptd' car nous avons déjà extrait les années, les mois et les jours dans de nouvelles colonnes. La conversion de 'date_rptd' en datetime servait seulement à remplir cet objectif d'extraction.

In [ ]:
#drop date_rptd
crime.drop('date_rptd', axis=1, inplace=True)

On peut enfin constaté qu'après notre data preprocesssing, nous n'avons aucune valeur manquante, ce qui est idéal pour une analyse fiable et efficace.

In [ ]:
crime.isna().sum()

DR_NO                  0
date_occ               0
time_occ               0
AREA                   0
area_name              0
rpt_dist_no            0
Part 1-2               0
crm_cd                 0
crm_cd_desc            0
Mocodes                0
vict_age               0
vict_sex               0
vict_descent           0
premis_cd              0
premis_desc            0
weapon_used_cd         0
weapon_desc            0
Status                 0
status_desc            0
LOCATION               0
cross_street      779952
LAT                    0
LON                    0
Année                  0
Mois                   0
Jours                  0
crime_category         0
crime_type             0
dtype: int64

In [ ]:
#visualtion de notre base de donnée nettoyé.
crime.head(5)

,DR_NO,date_occ,time_occ,AREA,area_name,rpt_dist_no,Part 1-2,crm_cd,crm_cd_desc,Mocodes,...,status_desc,LOCATION,cross_street,LAT,LON,Année,Mois,Jours,crime_category,crime_type
0,190326475,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,MVT,VEHICLE - STOLEN,1501 Other MO,...,Adult Arrest,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Larceny,Property
3,200907217,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,OTHER.THEFT,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,...,Invest Cont,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2023,5,10,Larceny,Property
4,220614831,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,OTHER.THEFT,THEFT OF IDENTITY,1822 1501 0930 2004,...,Invest Cont,1900 TRANSIENT,NaN,34.0944,-118.3277,2022,8,18,Larceny,Property
5,231808869,12/01/2020 12:00:00 AM,2300,18,Southeast,1826,2,OTHER.THEFT,THEFT OF IDENTITY,1822 0100 0930 0929,...,Invest Cont,9900 COMPTON AV,NaN,33.9467,-118.2463,2023,4,4,Larceny,Property
6,230110144,07/03/2020 12:00:00 AM,900,1,Central,182,2,OTHER.THEFT,THEFT OF IDENTITY,0930 0929,...,Invest Cont,1100 S GRAND AV,NaN,34.0415,-118.2620,2023,4,4,Larceny,Property


On enregistre le dataframe nettoyé dans un nouveau fichier CSV. index = false car nous voulons pas avoir de probleme d'importation et meme seulement pour économiser de l'espace de stockage.

In [ ]:
# 
crime_data = crime.to_csv('allcleanedcrimedata', index=False)